In [2]:
# import standard stuff...
import os
import sys
import cellml

First we want to parse the CellML document that we are going to annotate. We parse in non-strict mode to allow any version CellML model to be used.

In [3]:
model_file = 'models/sine_approximations.xml'
model = cellml.parse_model(model_file, False)

Now we create an annotator object that we can use to manage the metadata IDs on our model

In [4]:
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

Make sure all entities in the model have an ID and make sure that all IDs in the model are unique.

In [5]:
if annotator.assignAllIds():
    print('Some entities have been assigned an ID, you should save the model!')
else:
    print('Everything already had an ID.')

duplicates = annotator.duplicateIds()
if len(duplicates) > 0:
    print("There are some duplicate IDs, behaviour may be unreliable...")
    print(duplicates)

Some entities have been assigned an ID, you should save the model!
There are some duplicate IDs, behaviour may be unreliable...
('sin', 'x')


Since this test model has some messed up IDs, lets just blow them away and start again...

In [6]:
annotator.clearAllIds()
annotator.assignAllIds()
model_string = cellml.print_model(model)
print(model_string)

# and save the updated model to a new file - note, we need the model filename for making our annotations later
model_file = 'models/sine_approximations-updated-ids.xml'
with open(model_file, 'w') as f:
    f.write(model_string)

<?xml version="1.0" encoding="UTF-8"?>
<model xmlns="http://www.cellml.org/cellml/2.0#" name="sin_approximations_import" id="b4da6c">
  <component name="main" id="b4da6d">
    <variable name="x" units="dimensionless" initial_value="0" interface="public_and_private" id="b4da71"/>
    <variable name="sin1" units="dimensionless" interface="public_and_private" id="b4da72"/>
    <variable name="sin2" units="dimensionless" interface="public_and_private" id="b4da73"/>
    <variable name="deriv_approx_initial_value" units="dimensionless" initial_value="0" interface="public_and_private" id="b4da74"/>
    <variable name="sin3" units="dimensionless" interface="public_and_private" id="b4da75"/>
  </component>
  <component name="actual_sin" id="b4da6e">
    <variable name="x" units="dimensionless" interface="public_and_private" id="b4da76"/>
    <variable name="sin" units="dimensionless" interface="public_and_private" id="b4da77"/>
    <math xmlns="http://www.w3.org/1998/Math/MathML">
      <apply 

Now we can do some annotation using pyOmexMeta (see https://github.com/sys-bio/pyomexmeta-binder-notebook) - except it doesn't work for CellML 2.0 models. So for now we work directly with RDF.

In [13]:
# get the ID of the variables we want to annotate
actual_sine_variable = model.component('actual_sin').variable('sin').id()
output_variable = model.component('parabolic_approx_sin').variable('sin').id()

# make an RDF Graph to add annotations to - using rdflib
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import DCTERMS

g = Graph()

# define some URIs for things we need

# use this URI to identify delayed variables - not the perfect URI, but will do for now
#     This is actually "delayed differential equation model" from the MAMO ontology
delay_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000089')

# use this for some random thing that I want to define - http://example.com is a good base for things that will never resolve
stuff_uri = 'http://example.com/cool-thing#21'

# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')

# Create an RDF URI node for our variable to use as the subject for multiple triples
# note: we are going to serialise the RDF graph into the same folder, so we need a URI that is relative to the intended file
variable_uri = URIRef('sine_approximations-updated-ids.xml' + '#' + actual_sine_variable)
output_uri = URIRef('sine_approximations-updated-ids.xml' + '#' + output_variable)

# Add triples using store's add() method.
# We're using the Dublin Core term "type" to associate the variable with the delay...
g.add((variable_uri, DCTERMS.type, delay_variable_uri))
# Set a timecourse readout variable
g.add((output_uri, DCTERMS.type, timecourse_readout_uri))

# print all the data in the turtle format
print(g.serialize(format='ttl'))

# and save to a file
with open('models/sine_approximations-updated-ids--annotations.ttl', 'w') as f:
    f.write(g.serialize(format='ttl'))

@prefix ns1: <http://purl.org/dc/terms/> .

<sine_approximations-updated-ids.xml#b4da77> ns1:type <http://identifiers.org/mamo/MAMO_0000089> .

<sine_approximations-updated-ids.xml#b4da7c> ns1:type <http://identifiers.org/mamo/MAMO_0000031> .


